<a href="https://colab.research.google.com/github/alexcpn/tranformer_learn/blob/main/gpt2_overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.28.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.0 MB/s eta 0:00:00


In [2]:
#upload files to your colab environment
!wget https://raw.githubusercontent.com/alexcpn/tranformer_learn/main/data/small_3.txt

--2023-04-26 03:59:49--  https://raw.githubusercontent.com/alexcpn/tranformer_learn/main/data/small_3.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56513 (55K) [text/plain]
Saving to: ‘small_3.txt’

small_3.txt         100%[===================>]  55.19K  --.-KB/s    in 0.01s   

2023-04-26 03:59:50 (5.28 MB/s) - ‘small_3.txt’ saved [56513/56513]



In [3]:
train_path = 'small_3.txt'


In [4]:
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import AutoTokenizer

def load_dataset(path,tokenizer):
    dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=path,
          block_size=128)
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return dataset,data_collator

tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
train_dataset,data_collator = load_dataset(train_path,tokenizer)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (12020 > 1024). Running this sequence through the model will result in indexing errors


In [5]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2-medium")


training_args = TrainingArguments(
    output_dir="./gpt2-small3-v2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=50, # number of training epochs
    per_device_train_batch_size=2, # batch size for training
    per_device_eval_batch_size=2,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
)

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [6]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.263500
1000,0.205800
1500,0.029900


KeyboardInterrupt: ignored


Short detail about training - 
- Step	Training Loss
- 500	2.263500
- 1000	0.205800
- 1500	0.029900

Took about 6 hours in Colab Free in TPU runtime

Stopped at Epoch 35 as the Training loss was pretty low

1655/2350 7:15:15 < 3:03:00, 0.06 it/s, Epoch 35.19/50]

In [ ]:
trainer.save_model()

In [8]:
!zip -r gpt2-small3-v2.zip gpt2-small3-v2/checkpoint-1600/


  adding: gpt2-small3-v2/checkpoint-1600/ (stored 0%)
  adding: gpt2-small3-v2/checkpoint-1600/optimizer.pt (deflated 8%)
  adding: gpt2-small3-v2/checkpoint-1600/trainer_state.json (deflated 57%)
  adding: gpt2-small3-v2/checkpoint-1600/generation_config.json (deflated 24%)
  adding: gpt2-small3-v2/checkpoint-1600/rng_state.pth (deflated 23%)
  adding: gpt2-small3-v2/checkpoint-1600/config.json (deflated 52%)
  adding: gpt2-small3-v2/checkpoint-1600/training_args.bin (deflated 48%)
  adding: gpt2-small3-v2/checkpoint-1600/pytorch_model.bin (deflated 9%)
  adding: gpt2-small3-v2/checkpoint-1600/scheduler.pt (deflated 49%)


In [15]:
from google.colab import files
files.download('gpt2-small3-v2.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
!cp /content/gpt2-small3-v2.zip ./drive/MyDrive/Models

In [10]:
from transformers import pipeline

test = pipeline('text-generation',model='./gpt2-small3-v2/checkpoint-1600/', tokenizer='gpt2')

In [11]:
test('An alkaline medium favours', max_new_tokens=100)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'An alkaline medium favours bacterial growth; and moisture is a necessary condition; spores, however, can survive the want of water for much longer periods than fully developed bacteria. The necessity for oxygen varies in different species. Those that require oxygen are known as  aërobic bacilli  or  aërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes. The great majority of bacteria, however, while they prefer to have oxygen, are able to live without'}]

This is as  expected from the passage
[ An alkaline medium favours bacterial growth; and moisture is a necessary condition; spores, however, can survive the want of water for much longer periods than fully developed bacteria. The necessity for oxygen varies in different species. Those that require oxygen are known as  aërobic bacilli  or  aërobes ; those that cannot live in the presence of oxygen are spoken of as  anaërobes . The great majority of bacteria, however, while they prefer to have oxygen, are able to live without it, and are called  facultative anaërobes

In [12]:
test('Streptococci are met with in', max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig. 36). Some multiply by fission, others by sporulation. Some forms are motile, others are non-motile. Some forms are motile only in virtue of the contractility of the protoplasm, some in virtue of the fibroblasts which they carry. Others are'}]

The inital part is exaclty as in the passage [Streptococci are met with in erysipelas and various other inflammatory and suppurative processes of a spreading character.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig. 4). Some multiply by fission, others by sporulation. Some forms are motile, others are non-motile.] 

The last line

from passage [Tuberculosis, tetanus, anthrax, and many other surgical diseases are due to different forms of bacilli.  Spirilla  are long, slender, thread-like cells, more or less spiral or wavy.]

The last line generated is not correct

"Some forms are motile only in virtue of the contractility of the protoplasm, "

is there in another pace where "motile" is reffered

whereas the following is pure hallicunation

"some in virtue of the fibroblasts which they carry. Others are'}]"

In [13]:
test('What is Streptococci', max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'What is Streptococci? Streptococci are long, slender, thread-like cells, more or less spiral or wavy. Some move by a screw-like contraction of the protoplasm, some by flagellainternalisation. The spirochate associated with syphilis (Fig. 36) is the most important member of this group.  Clinical Use of Streptococci.  When a new disease is brought about by the action of an invading micro-organism, the initial shock of'}]

In [14]:
test('Streptococci', max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Streptococci  are met with in erysipel Legs and Upper Cutaneous Tissue.  Bacilli  are rod-shaped bacteria, usually at least twice as long as they are broad (Fig. 5Hypothesis 1). Some multiply by fission, others by sporulation. Some forms are motile, others are non-motile. Some forms are motile by the introduction of  spores, which are produced in the interior of the growing medium. Spores are remarkable for their tenacity'}]

In [15]:
 test('John Hilton\'s classical work', max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'John Hilton\'s classical work, "The Blood," which appeared in London in 1755, is the most important work of recent writers on the subject of blood. In it he distinguishes between the normal corpuscles, which are derived from the proliferating cells of the part, and the specialised corpuscles, which are derived from the surviving cells of the part. The great majority of writers do not recognise these varieties of corpuscles as distinct from one another, but as forms of temporary life-support for the developing cells of the'}]

In [17]:
 test('Who was Ghandi', max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Who was Ghandi?  A short biography of the great humanist, written by A. E. Wright, Esq. It would appear that the germ theory of disease is derived from the fact that certain pathological conditions, such as those of high blood pressure, diabetes, syphilis, scurvy, or alcoholism, sometimes present themselves in the course of old age, when the vitality of the tissues is low, and when the individual is in a normal health. The theory is advanced by A. E. Wright'}]

In [18]:
 test('skin grafting', max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'skin grafting is carried out when the surface is covered by a uniform layer of healthy granulations and before the inevitable contraction of scar tissue makes itself manifest. Before applying the grafts it is usual to scrape away the granulations until the young fibrous tissue underneath is exposed, but, if the granulations are healthy and can be rendered aseptic, the grafts may be placed on them directly. If it is decided to scrape away the granulations, the oozing must be arrested by pressure with a pad'}]